In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
# Install huggingface library
!pip install -U torch ray==2.6.3 transformers hyperopt accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import random
import numpy as np

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# read here your dataset
import pandas as pd

columns_to_read = ["Text", "greenyesno"]

# Load the dataset into a pandas dataframe.
#df = pd.read_csv("/content/gdrive/My Drive/bert/dataset/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
#df = pd.read_csv("/content/gdrive/My Drive/bert/dataset/eco_dataset.tsv", delimiter='\t', usecols=columns_to_read)
df = pd.read_csv("./eco_dataset_clean.tsv", delimiter='\t', usecols=columns_to_read)
#df = pd.read_csv("./eco_dataset_clean.tsv", delimiter='\t', usecols=columns_to_read)
df['greenyesno'] = df['greenyesno'].replace({'Eco-related': 1, 'Not eco-related': 0})
df.rename(columns={'Text': 'sentence'}, inplace=True)
df.rename(columns={'greenyesno': 'label'}, inplace=True)

In [ ]:
df.head()

,sentence,label
0,For environmentalists who want to reduce their...,1.0
1,🔴 Moscow’s mayor has urged residents to stay i...,0.0
2,"Who in their right mind spends £70,000+ on a c...",1.0
3,Shell ranks in the top 10 among the 90 compani...,1.0
4,"We're days away from #COP27, where world leade...",1.0


In [ ]:
# get here your sentences and labels
sentences = df.sentence.values
labels = df.label.values

In [ ]:
sentences[0]

'For environmentalists who want to reduce their own household waste without assigning themselves more chores, there is a simple option. This @Wirecutter-approved, easy-to-use container does the hard work of composting for you. [URL]'

In [ ]:
labels[0]

1.0

In [ ]:
# BERT tokenizer: To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary
from transformers import AutoTokenizer

huggingface_model_name = 'bert-base-uncased'
#huggingface_model_name = 'distilbert-base-uncased'

# Load the BERT tokenizer.
print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(huggingface_model_name, do_lower_case=True)  # it will download and save it in a cache local directory

Loading tokenizer...


In [ ]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
# Get the maximum length of the sentences in the dataset
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  223


In [ ]:
# let's encode the dataset
max_length = 128 # instead of 47, just in case there are some longer test sentences

input_ids = []
attention_masks = []
lab_tensor = torch.zeros((len(sentences), 2))

# For every sentence...
for i, sent in enumerate(sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_length,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True,
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

    # add label to lab_tensor
    if labels[i] == 0.0:
      lab_tensor[i, 0] = 1
    elif labels[i] == 1.0:
      lab_tensor[i, 1] = 1

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
#labels = torch.tensor(labels).long()    # Added here .long()

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
print('Labels:', lab_tensor[0])

Original:  For environmentalists who want to reduce their own household waste without assigning themselves more chores, there is a simple option. This @Wirecutter-approved, easy-to-use container does the hard work of composting for you. [URL]
Token IDs: tensor([  101,  2005,  4483,  5130,  2040,  2215,  2000,  5547,  2037,  2219,
         4398,  5949,  2302, 23911,  2075,  3209,  2062, 27091,  1010,  2045,
         2003,  1037,  3722,  5724,  1012,  2023,  1030,  7318, 12690,  3334,
         1011,  4844,  1010,  3733,  1011,  2000,  1011,  2224, 11661,  2515,
         1996,  2524,  2147,  1997,  4012, 19894,  2075,  2005,  2017,  1012,
         1031, 24471,  2140,  1033,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,  

In [ ]:
# the labels now are long
torch.set_printoptions(threshold=10)
labels

array([1., 0., 1., ..., 1., 1., 0.])

In [ ]:
import torch
from torch.utils.data import IterableDataset
from torch.utils.data import TensorDataset, random_split

class MyDataLoader(IterableDataset):

  def __init__(self, ids, mask, labels):
    super(MyDataLoader).__init__()
    self._ids = ids
    self._mask = mask
    self._labels = labels

  def __len__(self):
    return self._ids.size(dim=0)

  def __getitem__(self, idx):
    item = dict()
    item["input_ids"] = torch.Tensor(self._ids[idx])
    item["attention_mask"] = torch.Tensor(self._mask[idx])
    item["labels"] = self._labels[idx, :]
    return item

In [ ]:
# training and validation split - 90% train and 10% valid
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = MyDataLoader(input_ids, attention_masks, lab_tensor) #TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

2,523 training samples
  281 validation samples


In [ ]:
from transformers import EvalPrediction
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

def compute_metrics(p: EvalPrediction):
  y_true = p.label_ids
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
  y_pred = torch.zeros(preds.shape)
  args = torch.argmax(torch.Tensor(preds), dim=1)
  y_pred[:,args[:]] = 1
  precision = precision_score(y_true=y_true, y_pred=y_pred, average='micro')
  recall = recall_score(y_true=y_true, y_pred=y_pred, average='micro')
  f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
  roc_auc = roc_auc_score(y_true, y_pred, average='micro')
  accuracy = accuracy_score(y_true, y_pred)
  metrics = {'p': precision,
             'r': recall,
             'f1': f1_micro_average,
             'roc_auc': roc_auc,
             'accuracy': accuracy}
  return metrics

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Train with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )



In [ ]:
# define the model - we will use BERTForSequenceClassification because it has the same BERT architecture but with a single classification layer on top
from transformers import AutoModelForSequenceClassification

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.

def my_model_init():
  model = AutoModelForSequenceClassification.from_pretrained(        # use DistilBertForSequenceClassification if you want
      huggingface_model_name,
      num_labels = 2, # The number of output labels--2 for binary classification.
                      # You can increase this for multi-class tasks.
      output_attentions = False, # Whether the model returns attentions weights.
      output_hidden_states = False, # Whether the model returns all hidden-states.
      return_dict=True
  )

  for name, param in model.named_parameters():
    if 'Bert' in name:
      param.requires_grad = False

  model.to(device)

  return model

# move the model on cpu or on gpu
#model.to(device)

In [ ]:
# Freeze the BERT model parameters
#for name, param in model.named_parameters():
#  if 'Bert' in name:
#    param.requires_grad = False

In [ ]:
from contextlib import suppress
from transformers import Trainer, TrainingArguments
from ray import tune
#from ray.train import CheckpointConfig
from ray.tune import CLIReporter
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)
from ray.tune.schedulers import PopulationBasedTraining


batch_size = 16
num_epochs = 10

lr = 2e-5
eps= 1e-8
adam_beta_1 = 0.9
adam_beta_2 = 0.999
warmup_steps = len(train_dataset) * num_epochs



training_args = TrainingArguments(output_dir="trainer",
                                  overwrite_output_dir=True,
                                  do_train=True,
                                  do_eval=True,
                                  evaluation_strategy='epoch',
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  learning_rate=lr,
                                  adam_beta1=adam_beta_1,
                                  adam_beta2=adam_beta_2,
                                  adam_epsilon=eps,
                                  lr_scheduler_type='linear',
                                  warmup_steps=warmup_steps,
                                  num_train_epochs=num_epochs,
                                  save_strategy='epoch',
                                  save_total_limit=5,
                                  load_best_model_at_end=True,
                                  #metric_for_best_model='eval_accuracy',
                                  logging_strategy='epoch')


trainer = Trainer(
    model_init=my_model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

#trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    search_alg=HyperOptSearch(metric="objective", mode="max"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    scheduler=ASHAScheduler(metric="objective", mode="max"))

2023-09-25 12:54:53,980	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-09-25 12:54:54,004	WARNING callback.py:144 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2023-09-25 12:54:54,008	WARNING tune.py:997 -- AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


+-------------------------------------------------------------------+
| Configuration for experiment     _objective_2023-09-25_12-54-53   |
+-------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                  |
| Scheduler                        AsyncHyperBandScheduler          |
| Number of trials                 20                               |
+-------------------------------------------------------------------+

View detailed results here: /root/ray_results/_objective_2023-09-25_12-54-53

Trial status: 1 PENDING
Current time: 2023-09-25 12:54:54. Total running time: 0s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:V100)
+----------------------------------------------------------------------------------------------------------+
| Trial name            status       learning_rate     num_train_epochs      seed     ..._train_batch_size |
+---------------------------------------------------

(pid=3684) 2023-09-25 12:55:00.615401: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Trial _objective_6c47dcc5 started with configuration:
+----------------------------------------------+
| Trial _objective_6c47dcc5 config             |
+----------------------------------------------+
| learning_rate                              0 |
| num_train_epochs                           3 |
| per_device_train_batch_size               64 |
| seed                                 17.2138 |
+----------------------------------------------+



(_objective pid=3684) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
(_objective pid=3684) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/18 [00:00<?, ?it/s]
(_objective pid=3684) 
 22%|██▏       | 4/18 [00:00<00:00, 37.35it/s]


...

  0%|          | 0/18 [00:00<?, ?it/s]
(_objective pid=10271) 
 22%|██▏       | 4/18 [00:00<00:00, 38.37it/s]


(_objective pid=10271) {'loss': 0.7041, 'learning_rate': 4.25427549947295e-09, 'epoch': 1.0}


(_objective pid=10271) 
 44%|████▍     | 8/18 [00:00<00:00, 32.05it/s]
(_objective pid=10271) 
 67%|██████▋   | 12/18 [00:00<00:00, 30.37it/s]


Trial _objective_013b7f91 finished iteration 1 at 2023-09-25 13:19:32. Total running time: 24min 38s
+----------------------------------------------+
| Trial _objective_013b7f91 result             |
+----------------------------------------------+
| time_this_iter_s                     19.0132 |
| time_total_s                         19.0132 |
| training_iteration                         1 |
| epoch                                      1 |
| eval_accuracy                        0.48754 |
| eval_f1                              0.48754 |
| eval_loss                            0.70279 |
| eval_p                               0.48754 |
| eval_r                               0.48754 |
| eval_roc_auc                         0.48754 |
| eval_runtime                          0.6319 |
| eval_samples_per_second              444.714 |
| eval_steps_per_second                 28.487 |
| objective                            2.43772 |
+----------------------------------------------+

Trial _objective

(_objective pid=10271) 
 89%|████████▉ | 16/18 [00:00<00:00, 29.77it/s]
                                               
100%|██████████| 40/40 [00:16<00:00,  2.48it/s]


In [ ]:
best_trial

BestRun(run_id='c691cc98', objective=2.6666666666666665, hyperparameters={'learning_rate': 3.042747502045481e-05, 'num_train_epochs': 2, 'seed': 9.93058038171246, 'per_device_train_batch_size': 4}, run_summary=<ray.tune.analysis.experiment_analysis.ExperimentAnalysis object at 0x7c5c43c1ff70>)

In [ ]:
# Default objective is the sum of all metrics
# when metrics are provided, so we have to maximize it.
# trainer.hyperparameter_search(
#    direction="maximize",
#    backend="ray",
#    n_trials=10 # number of trials
#)

2023-09-21 17:07:02,639	INFO worker.py:1621 -- Started a local Ray instance.
2023-09-21 17:07:11,531	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2023-09-21 17:07:11,543	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
/usr/local/lib/python3.10/dist-packages/ray/tune/trainable/function_trainable.py:624: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/latest/tun

+-------------------------------------------------------------------+
| Configuration for experiment     _objective_2023-09-21_17-07-11   |
+-------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator            |
| Scheduler                        FIFOScheduler                    |
| Number of trials                 10                               |
+-------------------------------------------------------------------+

View detailed results here: /root/ray_results/_objective_2023-09-21_17-07-11

Trial status: 10 PENDING
Current time: 2023-09-21 17:07:11. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs
+--------------------------------------------------------------------------------------------------------------+
| Trial name               status       learning_rate     num_train_epochs       seed     ..._train_batch_size |
+------------------------------------------------------------------------------

(pid=6862) 2023-09-21 17:07:26.300626: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Trial _objective_4df1a_00000 started with configuration:
+-------------------------------------------------+
| Trial _objective_4df1a_00000 config             |
+-------------------------------------------------+
| learning_rate                             1e-05 |
| num_train_epochs                              5 |
| per_device_train_batch_size                  64 |
| seed                                    8.15396 |
+-------------------------------------------------+



(_objective pid=6862) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
(_objective pid=6862) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  2%|▏         | 4/215 [00:05<04:25,  1.26s/it]


...

(_objective pid=16214) {'loss': 0.7005, 'learning_rate': 5.030820593044931e-08, 'epoch': 2.0}


(_objective pid=16214) 
 11%|█         | 2/19 [00:00<00:00, 17.84it/s]
(_objective pid=16214) 
 21%|██        | 4/19 [00:00<00:01, 10.66it/s]
(_objective pid=16214) 
 32%|███▏      | 6/19 [00:00<00:01,  9.44it/s]
(_objective pid=16214) 
 42%|████▏     | 8/19 [00:00<00:01,  8.86it/s]
(_objective pid=16214) 
 47%|████▋     | 9/19 [00:00<00:01,  8.75it/s]
(_objective pid=16214) 
 53%|█████▎    | 10/19 [00:01<00:01,  8.64it/s]
(_objective pid=16214) 
 58%|█████▊    | 11/19 [00:01<00:00,  8.78it/s]
(_objective pid=16214) 
 63%|██████▎   | 12/19 [00:01<00:00,  8.67it/s]
(_objective pid=16214) 
 68%|██████▊   | 13/19 [00:01<00:00,  8.59it/s]
(_objective pid=16214) 
 74%|███████▎  | 14/19 [00:01<00:00,  8.66it/s]
(_objective pid=16214) 
 79%|███████▉  | 15/19 [00:01<00:00,  8.62it/s]
(_objective pid=16214) 
 84%|████████▍ | 16/19 [00:01<00:00,  8.33it/s]
(_objective pid=16214) 
 89%|████████▉ | 17/19 [00:01<00:00,  8.32it/s]
(_objective pid=16214) 
 95%|█████████▍| 18/19 [00:02<00:00,  8.28it/

Trial _objective_4df1a_00009 finished iteration 2 at 2023-09-21 17:46:49. Total running time: 39min 38s
+-------------------------------------------------+
| Trial _objective_4df1a_00009 result             |
+-------------------------------------------------+
| time_this_iter_s                        61.6286 |
| time_total_s                            124.151 |
| training_iteration                            2 |
| epoch                                         2 |
| eval_loss                               0.70483 |
| eval_runtime                             2.2515 |
| eval_samples_per_second                 134.576 |
| eval_steps_per_second                     8.439 |
| objective                               0.70483 |
+-------------------------------------------------+

(_objective pid=16214) {'eval_loss': 0.7048310041427612, 'eval_runtime': 2.2515, 'eval_samples_per_second': 134.576, 'eval_steps_per_second': 8.439, 'epoch': 2.0}
Trial _objective_4df1a_00009 completed after 2 iteration

BestRun(run_id='4df1a_00006', objective=0.7192806005477905, hyperparameters={'learning_rate': 1.6738085788752168e-05, 'num_train_epochs': 2, 'seed': 2.8199608653310015, 'per_device_train_batch_size': 32}, run_summary=<ray.tune.analysis.experiment_analysis.ExperimentAnalysis object at 0x7b78df854610>)

100%|██████████| 172/172 [02:15<00:00,  1.27it/s]


In [ ]:
# optimizer and lr scheduler - for finetuning, the authors of the original BERT paper recommend choosing from the following values:
# - Batch size: 16, 32  - we set the batch_size=32 when we created the dataloader
# Optimizer Adam
# Learning rate (lr): 5e-5, 3e-5, 2e-5  - we use lr = 2e-5
# Number of epochs: 2, 3, 4  - we will use 4

#optimizer = AdamW(model.parameters(),
#                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
#                  eps = 1e-8   # default is 1e-8
#                )

In [ ]:
#from transformers import get_linear_schedule_with_warmup

#epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
#total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
#scheduler = get_linear_schedule_with_warmup(optimizer,
#                                            num_warmup_steps = 0,
#                                            num_training_steps = total_steps)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# finetuning

training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU (or CPU) using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically
        optimizer.zero_grad()

        # Perform a forward pass
        # The documentation for this `model` function is here:
        # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        # The "logits" are the output values prior to applying an activation function like the softmax.
        loss, logits = model(b_input_ids,
                             attention_mask=b_input_mask,
                             labels=b_labels,
                             return_dict=False)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU (or CPU) using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            (loss, logits) = model(b_input_ids,
                                   attention_mask=b_input_mask,
                                   labels=b_labels,
                                   return_dict=False)

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
# Let’s view the summary of the training process

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

In [ ]:
# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

In [ ]:
# save the model and the tokenizer

import os

out_dir = '/content/gdrive/My Drive/eco_project_model/bert_base'

if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)

model.save_pretrained(out_dir)
tokenizer.save_pretrained(out_dir)